# Loading a saved model and merging the saved adapter weights
* this requires more memory than what the training takes up
* instance ml.g5.4xlarge
* make sure your libraries that are getting installed here match the ones you used at training stage

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops py7zr tensorboardX
!pip install -q -U torch==2.0.1 bitsandbytes==0.40.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.


In [3]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# GPU Check

In [5]:
!nvidia-smi

Mon Sep 25 04:08:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         Off  | 00000000:00:1E.0 Off |                    0 |
|  0%   26C    P8    17W / 300W |      2MiB / 22731MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Make sure the path is pointing to the correct checkpoint
* do not make any changes to the names of the files in the chek point direcotry

In [6]:
peft_model_id = "./results/checkpoint-10" # ensure this folder points to the right one
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path, 
    return_dict=True, 
    load_in_8bit=True, 
    device_map={"":0},
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [10]:
! mkdir merged_model_and_adapter_weights

In [ ]:
model = model.merge_and_unload()

In [13]:
torch.save(model.state_dict(), './merged_model_and_adapter_weights/merged_peft_model.pt')

In [15]:
! ls -la ./merged_model_and_adapter_weights

total 7052468
drwxr-xr-x 2 root root       6144 Sep 25 04:16 .
drwxr-xr-x 5 root root       6144 Sep 25 04:16 ..
-rw-r--r-- 1 root root 7221718205 Sep 25 04:18 merged_peft_model.pt
